### **The Diamond Problem and Method Resolution Order (MRO)**
This notebook demonstrates Python's Method Resolution Order (MRO) using the C3 Linearization algorithm. We will first analyze a successful cooperative inheritance pattern (the Diamond) and then examine a conflicting hierarchy that results in a TypeError.

Part 1: Cooperative Inheritance (The Diamond Pattern)

The Diamond pattern is a typical multiple inheritance scenario where a class (D) inherits from two classes (B and C) that share a common ancestor (A).

In [40]:
# Step 1: Basic class hierarchy to demonstrate MRO

class A:
    def __init__(self):
        print("A.__init__() called")

class B(A):
    def __init__(self):
        super().__init__()
        print("B.__init__() called")

class C(A):
    def __init__(self):
        super().__init__()
        print("C.__init__() called")

class D(B, C):
    def __init__(self):
        super().__init__()
        print("D.__init__() called")

In [52]:
# Create an instance of D
d = D()

# Find the MRO order
print("\n--- MRO Search Order ---")
print("D.mro() result:", D.mro())

A.__init__() called
C.__init__() called
B.__init__() called
D.__init__() called

--- MRO Search Order ---
D.mro() result: [<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]


In [ ]:
"""
The inheritance diagram looks like this:

    A
   / \
  B   C
   \ /
    D
"""

#### MRO Explanation: D(B, C)

The final MRO is D → B → C → A → object.

Priority 1: Class Precedence: The class itself (D) is always first.

Priority 2: Order Preservation (Left-to-Right): Because B is listed before C in D(B, C), the algorithm attempts to choose B next.

Priority 3: Monotonicity (Ancestors): This rule ensures that if a class has a parent, the class must appear before its parent in the MRO.

The algorithm chooses B next.

It then checks the remaining options: A (B's parent), C (D's second base), and A (C's parent).

A is temporarily blocked because it appears in the list of ancestors for C (i.e., it is in the "tail" of the lists being merged).

C is chosen next.

Finally, the common ancestor A is chosen, ensuring it is only visited once, after all its children (B and C) have been processed.

### **Step 2: Breaking MRO (Inconsistent Inheritance)**

In [49]:
class E(B, D):
    def __init__(self):
        super().__init__()
        
e = E()

TypeError: Cannot create a consistent method resolution
order (MRO) for bases B, D

In [ ]:
"""
MRO CONFLICT: WHY class E(B, D) FAILS (Simplified)

The failure is caused by an impossible contradiction in the required search order for methods. Python's Method Resolution Order (MRO) rules cannot satisfy both demands at the same time.

THE PROBLEM: Contradictory Rules

Python needs to place B and D in a single MRO list, but the existing structure imposes two opposite requirements:

RULE 1: E's Definition (Left-to-Right Precedence)
The line class E(B, D) forces the rule: B must be checked before D.
(Reason: Base classes are always searched from left to right.)

RULE 2: D's Existing MRO (Child Must Precede Parent)
Since D already inherits from B (D's MRO is [D, B, C...]), this rule enforces that D must be checked before B.
(Reason: A child class (D) must always appear before its parent (B) in any MRO.)

THE FAILURE: Total Blockage

The MRO algorithm encounters a deadlock:

It tries to choose B, but Rule 2 says D must come first. B is blocked.

It tries to choose D, but Rule 1 says B must come first. D is blocked.

THE RESULT: TypeError

Since both classes block each other, Python cannot establish a single, unambiguous search path. The process fails immediately, and the class E is never created.
"""

## The Best Possible Solution

The best solution is to eliminate the circular dependency and redundant inheritance by ensuring a class does not inherit from another class and from an ancestor that already contains that class in its MRO.

In the case of E(B, D), class D already inherits all attributes and methods from B. By having E inherit from both B and D, we create the conflict.

### **Solution:** Simplifying the Inheritance

The easiest fix is to remove the redundant parent B, allowing E to inherit B's behavior solely through D.

In [53]:
# The BEST POSSIBLE SOLUTION: Remove the redundant base class (B)
# Since D already inherits from B, E only needs to inherit from D.

class E_Fixed(D):
    def __init__(self):
        super().__init__() # This now correctly starts the MRO from D
        print("E_Fixed.__init__() called")

# Check the MRO for the fixed class
print("MRO for E_Fixed:", E_Fixed.mro())

# Demonstrate successful instantiation
e_fixed = E_Fixed()


MRO for E_Fixed: [<class '__main__.E_Fixed'>, <class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]
A.__init__() called
C.__init__() called
B.__init__() called
D.__init__() called
E_Fixed.__init__() called


The new MRO is: [<class '__main__.E_Fixed'>, <class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]

The conflict is resolved because the requirement for B > D is removed.

The established, non-conflicting MRO of $\mathbf{D} \rightarrow \mathbf{B} \rightarrow \mathbf{C} \rightarrow \mathbf{A}$ is preserved and simply prepended(put at the start) by $\mathbf{E\_Fixed}$.

All constructors are called exactly once in a predictable, linear order, fulfilling the goals of cooperative multiple inheritance.